In [33]:
import pandas as pd
import os
import re

In [34]:
# CALCULATE BUDGETS FOR EACH AREA
# link to source: https://app.clarity.so/rawdao/work/6

init_budget = {}

init_budget['governance'] = 2
init_budget['curation'] = 2.5
init_budget['marketing & pr'] = 2.5
init_budget['community'] = 1.5
init_budget['finance'] = 1.5

perf_tranche = {}

perf_tranche['governance'] =  0.4
perf_tranche['curation'] = 0.4
perf_tranche['marketing & pr'] = 0.4
perf_tranche['community'] = 0.5
perf_tranche['finance'] = 0.4

perf_score = {}

perf_score['governance'] =  5/6
perf_score['curation'] = 1/2
perf_score['marketing & pr'] = 2/3
perf_score['community'] = 3/3
perf_score['finance'] = 1/3


final_budget = {}
sum = 0
total_init = 0

for area in init_budget:
  final_budget[area] = round(init_budget[area]*(1-perf_tranche[area]) + init_budget[area]*(perf_tranche[area]*perf_score[area]),2)
  sum = sum + final_budget[area]
  total_init = total_init + init_budget[area]

print(final_budget)
print("total final budget = " + str(round(sum,2)) + " out of " + str(total_init))

{'governance': 1.87, 'curation': 2.0, 'marketing & pr': 2.17, 'community': 1.5, 'finance': 1.1}
total final budget = 8.64 out of 10.0


In [35]:
# READ AND CLEAN COORDINAPE DATA
dfs = {}
voting_members = {}
total_give = {}

for file in os.listdir('./coordinape_data'):
  try:
    # read data from coordinape files  
    key = re.search('DAO-(.+?)-', file).group(1).lower()
    # print(key)
    dfs[key] = pd.read_csv('./coordinape_data/'+ file)
    
    # calculate voting stats for each area
    voting_members[key] = int(dfs[key]['sent'].astype(bool).sum(axis=0))
    total_give[key] = int(dfs[key]['received'].sum())

    # calculate compensation per area
    dfs[key] = dfs[key].sort_values(by=['received'], ascending = False) # sort
    dfs[key] = dfs[key][['name','address','received']] # select relevant cols
    dfs[key]['percent_comp'] = dfs[key]['received']/total_give[key] # calculate percentage comp.
    dfs[key]['eth_compensation'] = dfs[key]['percent_comp'] * final_budget[key] # calculate actual ETH comp.
    dfs[key] = dfs[key][dfs[key]['eth_compensation'] > 0] # filter people with 0 compensation
    dfs[key]['area'] = key
    dfs[key]['total_budget_of_area'] = final_budget[key]
  except NameError:
    print(file + ' -> file skipped')
    print(key)
    print(NameError)

# prepare area breakdown for export
df_export = pd.concat(dfs)

# merge everything and calculate compensation per member
comp_df = pd.concat(dfs)
comp_df = comp_df[['name','address','eth_compensation']]
comp_df = comp_df.groupby(by=['name','address']).sum().reset_index()
comp_df = comp_df.sort_values(by=['eth_compensation'], ascending = False)

print('# of members who voted per area: ' + str(voting_members))
print('total votes per area: ' + str(total_give))
print("total compensated members: " + str(len(comp_df)))

# of members who voted per area: {'community': 17, 'curation': 17, 'finance': 8, 'governance': 16, 'marketing & pr': 19}
total votes per area: {'community': 1758, 'curation': 1692, 'finance': 755, 'governance': 1545, 'marketing & pr': 2069}
total compensated members: 44


In [36]:
# print votes per member per area
# pd.set_option('display.max_rows', None)
df_export[['name','total_budget_of_area','received','percent_comp','eth_compensation']]


name  total_budget_of_area  received  \
community      26           claudia#0273                  1.50       190   
               28  ezra | shibboleth.eth                  1.50       186   
               4             JoseCactuss                  1.50       170   
               6             Lapsus#4632                  1.50       161   
               34      matthewbrooks.eth                  1.50       149   
...                                  ...                   ...       ...   
marketing & pr 3                 HugoFaz                  2.17         5   
               27          ednz.eth#4780                  2.17         2   
               31             glory#5406                  2.17         2   
               11       MickeySmith#2765                  2.17         1   
               41       samedixsnft#8110                  2.17         1   

                   percent_comp  eth_compensation  
community      26      0.108077          0.162116  
               28      0.105802          0.158703  
               4       0.096701          0.145051  
               6       0.091581          0.137372  
               34      0.084755          0.127133  
...                         ...               ...  
marketing & pr 3       0.002417          0.005244  
               27      0.000967          0.002098  
               31      0.000967          0.002098  
               11      0.000483          0.001049  
               41      0.000483          0.001049  

[142 rows x 5 columns]

In [37]:
# print compensation per member
comp_df[['name','eth_compensation']]

,name,eth_compensation
25,claudia#0273,0.906549
32,lucaspon,0.721372
5,Lapsus#4632,0.628526
30,glory#5406,0.452348
33,matthewbrooks.eth,0.444038
...,...,...
9,MichaMandelberg#1138,0.016832
26,ednz.eth#4780,0.014003
7,M Ram,0.003631
43,thebonnie#3404,0.003631


In [38]:
# EXPORT RESULTS TO EXCEL
from pathlib import Path
Path("./outputs").mkdir(parents=True, exist_ok=True)

df_export.to_excel("./outputs/compensation_per_area.xlsx")
comp_df.to_excel("./outputs/compensation_by_member.xlsx")